PILAR ARIAS, YEDRA MARTÍN, PATRICIA MEDIAVILLA, AMAYA SARASA

<GRUPO 9>

#Práctica 1

##Introducción

Esta es la primera práctica del segundo laboratorio. Dada una base de datos en formato .csv, se nos pide contestar a una serie de cuestiones. Para ello, lo priemro que debemos hacer es cargar y leer la base de datos:

In [84]:
#Importar librerías
#!apt install mlxtend
import numpy as np #numpy
import pandas as pd #pandas
import sklearn as sk #scikit-learn
import matplotlib.pyplot as plt #gráfica Matplotlib
#from mlxtend.frequent_patterns import apriori
#from apyori import apriori #apriori
from mlxtend.preprocessing import TransactionEncoder
#from mlxtend.frequent_patterns import association_rules
from mlxtend.frequent_patterns import apriori, association_rules
#from mlxtend.frequent_patterns import frequent_patterns
#from mlxtend import frequent_patterns
#from mlxtend.frequent_patterns import frequent_patterns

#Importamos la base de datos
df = pd.read_csv('BlackFriday.csv',sep=',')
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 233599 entries, 0 to 233598
Data columns (total 11 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   User_ID                     233599 non-null  int64  
 1   Product_ID                  233599 non-null  object 
 2   Gender                      233599 non-null  object 
 3   Age                         233599 non-null  object 
 4   Occupation                  233599 non-null  int64  
 5   City_Category               233599 non-null  object 
 6   Stay_In_Current_City_Years  233599 non-null  object 
 7   Marital_Status              233599 non-null  int64  
 8   Product_Category_1          233599 non-null  int64  
 9   Product_Category_2          161255 non-null  float64
 10  Product_Category_3          71037 non-null   float64
dtypes: float64(2), int64(4), object(5)
memory usage: 19.6+ MB


,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3
0,1000004,P00128942,M,46-50,7,B,2,1,1,11.0,NaN
1,1000009,P00113442,M,26-35,17,C,0,0,3,5.0,NaN
2,1000010,P00288442,F,36-45,1,B,4+,1,5,14.0,NaN
3,1000010,P00145342,F,36-45,1,B,4+,1,4,9.0,NaN
4,1000011,P00053842,F,26-35,1,C,1,0,4,5.0,12.0


----------------

----------------

Subimos la base de datos, df.info nos aporta la infomación que contiene la base de datos, vemos el tipo, la cantidad de las variables con las que trabajamos... y .head vamos a ver que forma tiene la base de datos y las primeras filas de esta.

Según se ha estudiado en clase, encontramos dos métodos para afrontarlo. Nos decantaremos por CRISP-DM. Las siglas para: CRoss-Industry Standard Process for Data Mining. 

Es un modelo de proceso de minería de datos que se utiliza en la industria. Es un marco de trabajo que define un proceso estandarizado para llevar a cabo proyectos de minería de datos, desde la definición del problema hasta la implementación y evaluación del resultado. Los seis pasos que componen Crisp-DM son los siguientes:

- Comprensión de los negocios y definición del problema: comprender la naturaleza del problema y los objetivos de negocio.

- Comprensión de los datos: recopilación inicial, descripción y exploración  de datos.

- Preparación de los datos: Selección, limpieza, construcción, integración, formateo de los datos.

- Modelado: aplicar las técnicas de minería de datos a los dataset.

- Evaluación: determinar si los resultados son útiles a las necesidades del negocio.

- Despliegue: explotar utilidad de los modelos, integrándolos en las tareas de toma de decisiones de la organización -> Call to Action. 

##Comprensión de los negocios y definición del problema

La librería se usará de la siguiente manera:
1) Empezaremos obteniendo los itemsets frecuentes para k=1. En este punto necesitaremos obtener el
soporte de los itemset. Por lo tanto, se tendrá que usar un método que dado un itemset devuelva su
soporte.

2) Para k≥2.
a. Mostrar los itemset frecuentes candidatos y su soporte.

3) Repetir el proceso 2 hasta que no se generen nuevos itemsets frecuentes.

4) Mostrar todas las posibles reglas con la confianza de cada una de ellas.

5) Listar todas las reglas que sean de alta confianza.

6) Usar los siguientes métodos. Dado un antecesor, devolver todas las reglas que contengan a dicho
antecesor. Dado un umbral mínimo devolver todas las reglas que cumplan con dicha confianza.

7) Utiliza las representaciones gráficas que consideres adecuadas para representar las reglas obtenidas
y obtener conclusiones a partir de los datos.

A partir de lo anterior de pide:
- Prueba al menos tres configuraciones de soporte y frecuencia para cada género, edad y tipo de producto.
- ¿Qué diferencias hay entre usar soporte y frecuencia? Respalda la respuesta con datos 
- ¿Qué tipo reglas desaparecen según la configuración y categorías (género, edad y tipo de producto)
consideradas? ¿Por qué? 
- Para una de las configuraciones, interpreta algunas de las reglas que te hayan resultado interesantes
obtenidas usando la clase association_rules y un par de configuraciones cambiando la variable metric y
min_threshold. Justifica los resultados. 
- Dadas las mejores configuraciones. ¿Existen reglas o patrones que se repitan? ¿Podemos generalizar de
alguna manera como se comportan los clientes? Expón las conclusiones respaldadas con datos 

## Comprensión de los datos

La base de datos que utilizaremos consta de 233599 observaciones. Teniendo en ceunta que el trabajo que vamos a realizar, utilizanod el algoritmo apriori, queremos sacar regalas y con todos estos datos, lo que nos hace pensar es que vamos a encontrar muchísimas reglas a partir de estos datos.

La clave está en cuales son las que más nos interesan, basandonos en su confianza, y una vez hallado esto debemos tomar decisiones para que usando estas reglas vendamos el mayor número de producto, para aumentar nuestras gananzas.

Existen 11 tipos de atributos distintos: 

- User_ID: número para identificar al usuario.
- Product_ID: número para identificar el producto.
- Gender: Género del comprador.	
- Age: Edad del comprador.
- Occupation: A lo que se dedica el comprador, ya pasasdo a categórica.
- City_Category: Categoría de la ciudad donde reside el user.
- Stay_In_Current_City_Years: El tiempo que lleva en la ciudad.
- Marital_Status: Estado civil.
- Product_Category_1: La categoría principal del producto.
- Product_Category_2: Siguente categoría.
- Product_Category_3: Última categoría del producto.



##Preparación de los datos

In [85]:
usernan = df['User_ID'].isna().sum()
print(f"La cantidad de NaN que existen en la variable User_ID es:{usernan}")
productnan = df['Product_ID'].isna().sum()
print(f"La cantidad de NaN que existen en la variable Product_ID es:{productnan}")
gendernan = df['Gender'].isna().sum()
print(f"La cantidad de NaN que existen en la variable Gender es:{gendernan}")
agenan = df['Age'].isna().sum()
print(f"La cantidad de NaN que existen en la variable Age es:{agenan}")
ocupationan = df['Occupation'].isna().sum()
print(f"La cantidad de NaN que existen en la variable Occupation es:{ocupationan}")
City_Categorynan = df['City_Category'].isna().sum()
print(f"La cantidad de NaN que existen en la variable City_Category es:{City_Categorynan}")
Stay_In_Current_City_Yearsnan = df['Stay_In_Current_City_Years'].isna().sum()
print(f"La cantidad de NaN que existen en la variable Stay_In_Current_City_Years es:{Stay_In_Current_City_Yearsnan}")
Marital_Statusnan = df['Marital_Status'].isna().sum()
print(f"La cantidad de NaN que existen en la variable Marital_Status es:{Marital_Statusnan}")
Product_Category_1nan = df['Product_Category_1'].isna().sum()
print(f"La cantidad de NaN que existen en la variable Product_Category_1 es:{Product_Category_1nan}")
Product_Category_2nan = df['Product_Category_2'].isna().sum()
print(f"La cantidad de NaN que existen en la variable Product_Category_2 es:{Product_Category_2nan}")
Product_Category_3nan = df['Product_Category_3'].isna().sum()
print(f"La cantidad de NaN que existen en la variable Product_Category_3 es:{Product_Category_3nan}")

La cantidad de NaN que existen en la variable User_ID es:0
La cantidad de NaN que existen en la variable Product_ID es:0
La cantidad de NaN que existen en la variable Gender es:0
La cantidad de NaN que existen en la variable Age es:0
La cantidad de NaN que existen en la variable Occupation es:0
La cantidad de NaN que existen en la variable City_Category es:0
La cantidad de NaN que existen en la variable Stay_In_Current_City_Years es:0
La cantidad de NaN que existen en la variable Marital_Status es:0
La cantidad de NaN que existen en la variable Product_Category_1 es:0
La cantidad de NaN que existen en la variable Product_Category_2 es:72344
La cantidad de NaN que existen en la variable Product_Category_3 es:162562


In [86]:
# Rellenamos los valores na con 0, ya que no tiene categoría
df = df.fillna(0)

# Quitamos los duplicados
df = df.drop_duplicates()

##Modelado

In [87]:
df_User_Product = (df
        .groupby(['User_ID', 'Product_ID'])['Product_Category_1']
        .sum()
        .apply(lambda x:x>0)
        .unstack()
        .fillna(False)
        .reset_index()
        .set_index('User_ID')
          )

In [88]:
df_Gender = (df
            .groupby(['User_ID','Gender'])['Product_Category_1']
            .sum()
            .apply(lambda x: x>0)
            .unstack()
            .fillna(False)
            .reset_index()
            .set_index('User_ID')
)

In [89]:
df_Age = (df
        .groupby(['User_ID','Age'])['Product_Category_1']
        .sum()
        .apply(lambda x: x>0)
        .unstack()
        .fillna(False)
        .reset_index()
        .set_index('User_ID')
)

In [90]:
df_Occupation = (df
                .groupby(['User_ID','Occupation'])['Product_Category_1']
                .sum()
                .apply(lambda x: x>0)
                .unstack()
                .fillna(False)
                .reset_index()
                .set_index('User_ID')
)

In [91]:
df_City = (df
          .groupby(['User_ID','City_Category'])['Product_Category_1']
          .sum()
          .apply(lambda x: x>0)
          .unstack()
          .fillna(False)
          .reset_index()
          .set_index('User_ID')
)

In [92]:
df_Stay = (df
           .groupby(['User_ID','Stay_In_Current_City_Years'])['Product_Category_1']
           .sum()
           .apply(lambda x: x>0)
           .unstack()
           .fillna(False)
           .reset_index()
           .set_index('User_ID')
)

In [93]:
df_Marital = (df
              .groupby(['User_ID','Marital_Status'])['Product_Category_1']
              .sum()
              .apply(lambda x: x>0)
              .unstack()
              .fillna(False)
              .reset_index()
              .set_index('User_ID')
)

In [94]:
df_Product_Category_1 = (df
                         .groupby(['User_ID','Product_Category_1'])['Product_Category_2']
                         .sum()
                         .apply(lambda x: x>0)
                         .unstack()
                         .fillna(False)
                         .reset_index()
                         .set_index('User_ID')
)

In [95]:
df_Product_Category_2 = (df
                         .groupby(['User_ID','Product_Category_2'])['Product_Category_1']
                         .sum()
                         .apply(lambda x: x>0)
                         .unstack()
                         .fillna(False)
                         .reset_index()
                         .set_index('User_ID')
)

In [96]:
df_Product_Category_3 = (df
                         .groupby(['User_ID','Product_Category_3'])['Product_Category_1']
                         .sum()
                         .apply(lambda x: x>0)
                         .unstack()
                         .fillna(False)
                         .reset_index()
                         .set_index('User_ID')
)

In [115]:
df_Product_Category_3.value_counts()

0.0   3.0    4.0    5.0    6.0    8.0    9.0    10.0   11.0   12.0   13.0   14.0   15.0   16.0   17.0   18.0 
True  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False    246
                                                                                          True   False  False    109
                                                                                          False  True   False     87
                                                                            True   False  False  False  False     69
                                                                            False  True   True   False  False     61
                                                                                                                ... 
                    True   False  False  True   True   False  True   False  False  True   True   False  True       1
                                                                       

In [112]:
df_Product_Category_1=df_Product_Category_1.set_axis(['1_PC1','2_PC1','3_PC1','4_PC1','5_PC1','6_PC1','7_PC1','8_PC1','9_PC1','10_PC1','11_PC1','12_PC1','13_PC1','14_PC1','15_PC1','16_PC1','17_PC1','18_PC1'], axis=1)

In [114]:
df_Product_Category_2=df_Product_Category_1.set_axis(['0_PC2','2_PC2','3_PC2','4_PC2','5_PC2','6_PC2','7_PC2','8_PC2','9_PC2','10_PC2','11_PC2','12_PC2','13_PC2','14_PC2','15_PC2','16_PC2','17_PC2','18_PC2'], axis=1)

In [117]:
df_Product_Category_3=df_Product_Category_3.set_axis(['0_PC3','3_PC3','4_PC3','5_PC3','6_PC3','8_PC3','9_PC3','10_PC3','11_PC3','12_PC3','13_PC3','14_PC3','15_PC3','16_PC3','17_PC3','18_PC3'],axis=1)

In [110]:
df_Marital=df_Marital.set_axis(['0_Marital', '1_Marital'], axis=1)

In [103]:
df_Occupation=df_Occupation.set_axis(['0_Occ','1_Occ', '2_Occ', '3_Occ', '4_Occ', '5_Occ', '6_Occ', '7_Occ', '8_Occ', '9_Occ', '10_Occ', '11_Occ', '12_Occ', '13_Occ', '14_Occ', '15_Occ', '16_Occ', '17_Occ', '18_Occ', '19_Occ', '20_Occ'], axis=1)

In [107]:
df_Stay=df_Stay.set_axis(['0_Stay','1_Stay','2_Stay','3_Stay','4+_Stay'],axis=1)

In [118]:
data = pd.concat([df_User_Product, df_Occupation, df_City, df_Stay, df_Marital, df_Product_Category_1, df_Product_Category_2, df_Product_Category_3, df_Age, df_Gender], axis = 1)
data

,P00000142,P00000242,P00000342,P00000442,P00000542,P00000642,P00000742,P00000842,P00000942,P00001042,...,18_PC3,0-17,18-25,26-35,36-45,46-50,51-55,55+,F,M
User_ID,,,,,,,,,,,,,,,,,,,,,
1000001,False,False,False,False,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,True,False
1000002,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,True
1000003,False,False,False,False,False,False,False,False,False,False,...,True,False,False,True,False,False,False,False,False,True
1000004,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,True
1000005,False,False,False,False,False,True,False,False,False,False,...,False,False,False,True,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1006036,False,False,False,False,False,False,False,False,False,False,...,True,False,False,True,False,False,False,False,True,False
1006037,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,True,False
1006038,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,True,False


In [ ]:
print(data.shape)

In [119]:
#Esto es para k=1 // use_colnames = True, max_len = 3
#frq_items = apriori(df, min_support = 0.05, use_colnames = True, max_len = 3)

frequent_itemsets = apriori(data, min_support=0.05, use_colnames=True, max_len=3)
frequent_itemsets

,support,itemsets
0,0.082159,(P00000142)
1,0.052792,(P00002142)
2,0.058903,(P00003242)
3,0.066033,(P00003442)
4,0.053641,(P00003942)
...,...,...
26551,0.074181,"(18-25, M, 17_PC3)"
26552,0.051774,"(26-35, 17_PC3, F)"
26553,0.150399,"(26-35, 17_PC3, M)"
26554,0.081310,"(36-45, 17_PC3, M)"


In [ ]:
type(frequent_itemsets)

In [ ]:
#Para k=2

frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets2=frequent_itemsets[frequent_itemsets['length'] == 3]
frequent_itemsets2.head()

In [ ]:
#Para k=3
frequent_itemsets3=frequent_itemsets[frequent_itemsets['length'] == 1]
frequent_itemsets3

In [120]:
#Reglas de asociación
rules = association_rules(frequent_itemsets, metric='confidence', min_threshold=0.8)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(P00000142),(1_PC1),0.082159,0.905958,0.077406,0.942149,1.039947,0.002973,1.625579
1,(P00000142),(3_PC1),0.082159,0.485317,0.082159,1.000000,2.060511,0.042286,inf
2,(P00000142),(5_PC1),0.082159,0.852996,0.078085,0.950413,1.114206,0.008004,2.964579
3,(P00000142),(8_PC1),0.082159,0.767951,0.070107,0.853306,1.111146,0.007013,1.581854
4,(P00000142),(0_PC2),0.082159,0.905958,0.077406,0.942149,1.039947,0.002973,1.625579
...,...,...,...,...,...,...,...,...,...
31344,"(18_PC3, 26-35)",(16_PC3),0.085215,0.670175,0.075539,0.886454,1.322721,0.018430,2.904776
31345,"(18_PC3, M)",(16_PC3),0.193006,0.670175,0.165676,0.858399,1.280859,0.036329,2.329262
31346,"(18_PC3, 16_PC3)",(M),0.202852,0.717196,0.165676,0.816736,1.138792,0.020192,1.543156
31347,"(18_PC3, 17_PC3)",(M),0.183670,0.717196,0.149211,0.812384,1.132724,0.017483,1.507361


Pero tenemos que quitar aquellas reglas que tengan un 0 en el antecedente o en la consecuencia. 

In [121]:
mask = (rules['antecedents'].apply(lambda x: 0 in x) | rules['consequents'].apply(lambda x: 0 in x))

rules_with_zero = rules[mask]

In [122]:
rules = rules.drop(rules_with_zero.index)

In [123]:
#vemos reglas de alta confianza
rulesaltaconfianza = association_rules(frequent_itemsets, metric='confidence', min_threshold=0.95)
rulesaltaconfianza

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(P00000142),(3_PC1),0.082159,0.485317,0.082159,1.000000,2.060511,0.042286,inf
1,(P00000142),(5_PC1),0.082159,0.852996,0.078085,0.950413,1.114206,0.008004,2.964579
2,(P00000142),(3_PC2),0.082159,0.485317,0.082159,1.000000,2.060511,0.042286,inf
3,(P00000142),(5_PC2),0.082159,0.852996,0.078085,0.950413,1.114206,0.008004,2.964579
4,(P00000142),(0_PC3),0.082159,0.997963,0.082159,1.000000,1.002041,0.000167,inf
...,...,...,...,...,...,...,...,...,...
15064,"(14_PC3, 11_PC3)",(16_PC3),0.089119,0.670175,0.086233,0.967619,1.443831,0.026508,10.185777
15065,"(15_PC3, 11_PC3)",(16_PC3),0.099644,0.670175,0.094721,0.950596,1.418430,0.027942,6.676116
15066,"(18_PC3, 11_PC3)",(15_PC3),0.054490,0.565099,0.052113,0.956386,1.692422,0.021321,9.971652
15067,"(17_PC3, 11_PC3)",(16_PC3),0.088100,0.670175,0.084705,0.961464,1.434647,0.025663,8.558963


NO BORRAR: 

In [ ]:
#frequent_itemsets = apriori(data, min_support=0.6, use_colnames=True)
#frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
#frequent_itemsets1=frequent_itemsets[frequent_itemsets['length'] == 1]

In [ ]:
#frequent_itemsets[ (frequent_itemsets['length'] >1 ) & #esto sería para k=2
                   #(frequent_itemsets['support'] >= 0.8) ]

Vale, hay un apartado que pide representar gráficamente las reglas: 

In [80]:
plot_confidence_vs_support(rules)

NameError: ignored

##Evaluación

##Despliegue